In [1]:
import json
args = (str("--output_dir /mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/toy --model_name_or_path /mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/saved_model_v100 --train_data /opt/tiger/toy_data.jsonl --learning_rate 1e-5 --fp16 --num_train_epochs 3 --per_device_train_batch_size 4 --gradient_accumulation_steps 4 --dataloader_drop_last --train_group_size 16 --max_len 512 --weight_decay 0.01 --logging_steps 10 --save_strategy epoch --save_steps 1 --save_total_limit 3".split(" ")).replace("\'","\""))
print(args)

["--output_dir", "/mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/toy", "--model_name_or_path", "/mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/saved_model_v100", "--train_data", "/opt/tiger/toy_data.jsonl", "--learning_rate", "1e-5", "--fp16", "--num_train_epochs", "3", "--per_device_train_batch_size", "4", "--gradient_accumulation_steps", "4", "--dataloader_drop_last", "--train_group_size", "16", "--max_len", "512", "--weight_decay", "0.01", "--logging_steps", "10", "--save_strategy", "epoch", "--save_steps", "1", "--save_total_limit", "3"]


In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModel
from transformers import TextGenerationPipeline, AutoModelForCausalLM, LlamaTokenizerFast
import os

In [4]:
config = transformers.AutoConfig.from_pretrained("/mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/encoder_simcse_group15_batch1_a100_bge_manner/checkpoint-8000")
config.use_pooler_layer = True
# config.loss_type = model_args.loss_type
# config.num_labels = model_args.num_labels
# config.margin = model_args.margin
# logging(f'Loss Function: {config.loss_type}')
print('###config: ', config)
# model = AutoModelForCausalLM.from_pretrained("/mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/encoder_simcse_group15_batch1_a100_bge_manner/checkpoint-8000", device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained("/mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/encoder_simcse_group15_batch1_a100_bge_manner/checkpoint-8000")

###config:  RobertaConfig {
  "_name_or_path": "/mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/encoder_simcse_group15_batch1_a100_bge_manner/checkpoint-8000",
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.40.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "use_pooler_layer": true,
  "vocab_size": 250002
}



In [7]:
model = AutoModel.from_pretrained(
    "/mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/encoder_simcse_group15_batch1_a100_bge_manner/checkpoint-8000",
    config=config
)
tokenizer = AutoTokenizer.from_pretrained("/mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/encoder_simcse_group15_batch1_a100_bge_manner/checkpoint-8000")

In [6]:
print(model)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
     

In [10]:
query = tokenizer("hello world", return_tensors="pt")
print(query)
output = model(**query)

{'input_ids': tensor([[    0, 33600,    31,  8999,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


In [13]:
print(output.keys())
print(output.pooler_output)

odict_keys(['last_hidden_state', 'pooler_output'])
tensor([[-0.4685, -0.1635, -0.0385,  ..., -0.1976,  0.1712,  0.0859]],
       grad_fn=<TanhBackward0>)


In [14]:
model1 = AutoModel.from_pretrained(
    "/mnt/bn/data-tns-live-llm/leon/experiments/llm/fcbank/encoder_simcse_group15_batch1_a100_bge_manner/checkpoint-8000"
)
output1 = model1(**query)
print(output1.keys())
print(output.pooler_output)

odict_keys(['last_hidden_state', 'pooler_output'])
tensor([[-0.4685, -0.1635, -0.0385,  ..., -0.1976,  0.1712,  0.0859]],
       grad_fn=<TanhBackward0>)
